In [3]:
from sklearn import svm
import numpy as np
from scipy.io import loadmat
from sklearn.svm import SVC
from RVM import RVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import KFold

global Parameters
Params={
    'SVM':{
        'gamma' : [1]
    },
    'RVM':{
        'alpha':[1e-06],
        'beta' : [1e-06]
    },
    'GPR':{
        'length_scale' : [10]
    }
}

In [4]:
def PreProcessing(file_name):
    '''
    target = loadmat(file_name)["Proj2TargetOutputsSet1"]
    m = len(target)
    n = len(target[0])
    y = []
    for i in xrange(m):
        for j in xrange(n):
            if target[i][j] == 1:
                y.append(j)
    return y
    '''
    target = loadmat(file_name)["Proj2TargetOutputsSet1"]
    number_labels = []
    for ars in target:
        if np.all(ars == [1, -1, -1, -1, -1]):
            ars = 1
            number_labels.append(ars)
        elif np.all(ars == [-1, 1, -1, -1, -1]):
            ars = 2
            number_labels.append(ars)
        elif np.all(ars == [-1, -1, 1, -1, -1]):
            ars = 3
            number_labels.append(ars)
        elif np.all(ars == [-1, -1, -1, 1, -1]):
            ars = 4
            number_labels.append(ars)
        elif np.all(ars == [-1, -1, -1, -1, 1]):
            ars = 5
            number_labels.append(ars)

    return np.asarray(number_labels)

In [5]:
def SVMTraining(XEstimate,XValidate,Parameters,class_labels):
    #clf = svm.SVC(decision_function_shape='ovo',Parameters)
    svcClassifier = SVC(kernel='rbf',  probability=True)
    gridSearcher = GridSearchCV(svcClassifier, Parameters)
    clf = OneVsRestClassifier(gridSearcher)
    #clf = OneVsRestClassifier(GridSearchCV(SVC(kernel='rbf',  probability=True), Parameters))
    print(clf.fit(XEstimate, class_labels))
    Yvalidate=clf.predict(XValidate)
    EstParameters=clf.get_params()
    print(clf.predict_proba(XValidate))
    mini = 1
    for i in clf.predict_proba(XValidate):
        mini = min(max(i), mini)
    print(mini)
    print(svcClassifier.__dict__)
    #print(clf.d(XValidate))
    return {"Yvalidate": Yvalidate,
            "EstParameters": EstParameters,
            "clf": clf}

In [6]:
def RVMTraining(XEstimate,XValidate,Parameters,class_labels):
    clf = OneVsOneClassifier(GridSearchCV(RVC(kernel='rbf', n_iter=1), Parameters))
    print(clf.fit(XEstimate, class_labels))
    Yvalidate = clf.predict(XValidate)
    EstParameters = clf.get_params()
    return {"Yvalidate": Yvalidate,
            "EstParameters": EstParameters,
            "clf": clf}

In [7]:
def GPRTraining(XEstimate,XValidate,Parameters,class_labels):
    kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-05, 100000.0))
    #clf = GaussianProcessClassifier(kernel=kernel, n_restarts_optimizer=1)
    clf = GaussianProcessClassifier(kernel= RBF(length_scale=1.0), optimizer=None,
                                       multi_class='one_vs_one', n_jobs=1)

    clf.fit(XEstimate, class_labels)
    Yvalidate = clf.predict(XValidate)
    EstParameters = clf.get_params()
    
    return {"Yvalidate": Yvalidate,
            "EstParameters": EstParameters,
            "clf": clf}

In [10]:
def MyCrossValidate(XTrain, Nf, ClassLabels):
    appended = list(zip(XTrain, ClassLabels))

    
    kf = KFold(n_splits=Nf)

    ConfMatrix = np.zeros([len(unique_number_of_class_labels), len(unique_number_of_class_labels)], dtype=int)
    EstConfMatrices = []
    for train, test in kf.split(appended):
        if Parameters == "SVM":
            res = SVMTraining(train[:][0:len(train[0])-1], train[:][-1], Params[Parameters], test[:][0:len(train[0])-1])
        elif Parameters == "RVM":
            res = RVMTraining(train[:][0:len(train[0])-1], train[:][-1], Params[Parameters], test[:][0:len(train[0])-1])
        elif Parameters == "GPR":
            res = GPRTraining(train[:][0:len(train[0])-1], train[:][-1], Params['GPR'], test[:][0:len(train[0])-1])
        
        l = MyConfusionMatrix(res["Yvalidate"], " ",test[:][-1])
        EstConfMatrices.append(l[0])
        ConfMatrix += l[0]

In [11]:
def MyConfusionMatrix(predictedLabels, ClassNames, actualLabels):
    unique_number_of_class_labels = sorted(set(actualLabels))
    matrix = np.zeros([len(unique_number_of_class_labels), len(unique_number_of_class_labels)], dtype=int)
    correctHits = 0
    for i in range(len(actualLabels)):
        matrix[actualLabels[i]][predictedLabels[i]] += 1
        if actualLabels[i] == predictedLabels[i]:
            correctHits += 1
    
    confusion_matrix = matrix[:][:]

    confusion_matrix2 = matrix / matrix.sum(axis=1)[:, None]


    print(confusion_matrix2)
    
    return (confusion_matrix, correctHits/len(actualLabels))

In [12]:
def TestMyClassifier(XTest, Parameters, EstParameters):
    Ytest = clf.predict(XTest)
    return Ytest

In [13]:
def TrainMyClassifier(XEstimate,XValidate,Parameters,class_labels):
    if Parameters == "SVM":
        return SVMTraining(XEstimate,XValidate,Params[Parameters],class_labels)
    elif Parameters == "RVM":
        return RVMTraining(XEstimate,XValidate,Params[Parameters],class_labels)
    elif Parameters == "GPR":
        return GPRTraining(XEstimate,XValidate,Params[Parameters],class_labels)
    

In [ ]:
if __name__ == '__main__':
    x=loadmat("Proj2FeatVecsSet1.mat")["Proj2FeatVecsSet1"]
    y=PreProcessing("Proj2TargetOutputsSet1.mat")
    y = np.asarray(y)
    pca = PCA(n_components=10)
    reduced_XEstimate = pca.fit_transform(x)
    l=[]
    t= []
    for x in range(1,25000,10):
        l.append(x)
    xe=[]
    yy=[]
    testData = []
    testLabels = []
    #for i in range(10):
    #    testData.append(100)
    for i in l:
        xe.append(reduced_XEstimate[i])
        yy.append(y[i])
        testData.append(reduced_XEstimate[i-1])
        testLabels.append(y[i-1])

    #should load the test data here
    #RVMTraining(reduced_XEstimate[10000:19000], reduced_XEstimate[100:200], Params['RVM'],y[10000:19000])
    #GPRTraining(reduced_XEstimate, reduced_XEstimate, Params['GPR'],y)
    #print(xe[0])
    print("Input the classifier you want to train (RVM / SVM / GPR) :")
    Parameters = input()
    res = TrainMyClassifier(xe, testData, Parameters ,yy)
    MyCrossValidate(xe, 5, yy)
    print(res["Yvalidate"])


Input the classifier you want to train (RVM / SVM / GPR) :
